# Exercise 04: Bayesian Inverse Problems

In this notebook we finally get started with Bayesian inverse problems. In particular, we see how to define likelihoods, priors and posteriors and how to sample them using CUQIpy.

## Learning objectives
* Define posterior distrubition in CUQIpy.
* Compute estimates of posterior (ML, MAP).
* Sample posterior with different samplers and compare results.
* See how the high-level BayesianProblem essentially wraps the above points for convinience.

## Table of contents
* [1. Defining the posterior distribution](#posterior)
* [2. Computing point estimates of posterior](#pointestimates)
* [3. Posterior sampling](#sampling)
* [4. Connection to BayesianProblem](#BayesianProbem)


## 1. Defining the posterior distribution

As before, we first import the packages we need

In [ ]:
import numpy as np
import cuqi

%load_ext autoreload
%autoreload 2

### Model and data
For this example let us revisit the Deconvolution testproblem and extract a CUQIpy model and some data. Similar to earlier, we can also get additional information from the 3rd output argument (probInfo).

In [ ]:
model, data, probInfo = cuqi.testproblem.Deconvolution.get_components()
n = model.range_dim
probInfo

### Likelihood
From the problem info string above, we see that the noise is additive Gaussian. Hence, we need to define a Gaussian likelihood, with the model as mean. This is easily done as follows.

In [ ]:
likelihood = cuqi.distribution.Gaussian(model,0.05)

Note in particular that likelihood is a conditional distribution. Conditional on the input parameter to the model (in this case x). This can be seen by checking

In [ ]:
likelihood.get_conditioning_variables()

#### Try yourself (optional):  
Try computing some relalizations of the noise. 

**Hint:** What is $\mathbf{x}$ if we are only interested in the noise?

In [ ]:
# This is where you type the code:




### Prior

In this case, let us assume the ground truth is well-represented by a Gaussian distrubiton. Hence we can define a Gaussian CUQIpy distrubiton as the prior.

In [ ]:
prior = cuqi.distribution.Gaussian(np.zeros(n),0.1)

### Combine into posterior
Once we have the likelihood, prior and an observed set of data we can define the posterior distribution

In [ ]:
posterior = cuqi.distribution.Posterior(likelihood,prior,data)

## ML estimation

In [ ]:
help(posterior.loglikelihood_function)

In [ ]:
solver_ML = cuqi.solver.maximize(posterior.loglikelihood_function,np.zeros(n))
x_ML, info = solver_ML.solve()

In [ ]:
x_ML = cuqi.samples.CUQIarray(x_ML,geometry=cuqi.geometry.Continuous1D(n))
x_ML.plot()

## MAP estimation

In [ ]:
MAP = cuqi.solver.maximize(posterior.logpdf,np.zeros(n))
x_map,info = MAP.solve()
x_map = cuqi.samples.CUQIarray(x_map,geometry=cuqi.geometry.Continuous1D(n))
x_map.plot()

## Posterior sampling

In [ ]:
sampler = cuqi.sampler.pCN(posterior,scale=0.1)
samples,_,_ = sampler.sample_adapt(10000,0)

In [ ]:
samples.plot_chain(60)

In [ ]:
samples.burnthin(6000,5).plot_ci(95)

## Posterior sampling (another better sampler)

In [ ]:
sampler = cuqi.sampler.Linear_RTO(likelihood,prior,model,data,np.zeros(n))
samples = sampler.sample(10000,0)

In [ ]:
samples.plot_chain(60)

In [ ]:
samples.burnthin(1000).plot_ci(95)

## Using geometries to parametrize x and improve pCN sampling result.

## High-level interface

In [ ]:
BP = cuqi.problem.BayesianProblem(likelihood,prior,data)

In [ ]:
samples = BP.sample_posterior(5000)

In [ ]:
samples.plot_ci(95,exact=probInfo.exactSolution)

In [ ]:
x_sol = BP.ML()[0]
x_sol = cuqi.samples.CUQIarray(x_sol,geometry=cuqi.geometry.Continuous1D(n))
x_sol.plot()

In [ ]:
x_map = BP.MAP()
x_map.plot()